# python code for results from Forster et al., 2025

Title: "Pre-stimulus beta power encodes explicit and implicit perceptual biases in distinct cortical areas"

notebook loads functions for behavioral and EEG analysis and reproduces figure 1,2,3 and 4

figure 5 and 6 are based on Rscripts, which are used for regression and mediation analysis

results are published in Forster et al., 2025
___

    Author:  Carina Forster et al.
    Contact: forster@cbs.mpg.de
    Years:   2024

___

Make sure you are in the right environment: `expecon_3.9`

## Setup 

### Imports

In [ ]:
# turn off warnings for a cleaner output
import warnings

warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import packages
from pathlib import Path

import matplotlib.pyplot as plt
import mne
import pandas as pd

# import functions for behavioral analysis
from expecon_ms.behav import figure1 as behav

# expecon_ms functions
from expecon_ms.configs import config, params, paths
from expecon_ms.eeg.preprocessing import ica

# Import functions from expecon_package for preproccesing eeg data
from expecon_ms.eeg.preprocessing import prepro as pp

# import functions for EEG analysis and visualization
from expecon_ms.eeg.sensor import evokeds as evo
from expecon_ms.eeg.sensor import tfr_contrasts as tfr

# import functions for source analysis
#from expecon_ms.eeg.source import source_reco
from scipy.stats import wilcoxon

### Set vars, paths, & constants

In [ ]:
# Define the output for mne functions
mne.set_log_level("CRITICAL")

## Analyse 

### 1. Behavioral data analysis (Signal detection theory based)

In [ ]:
# check the function arguments the docs
help(behav.plot_figure1_grid)

In [ ]:
behav.plot_figure1_grid(expecon=1, exclude_high_fa=True)

### 2. Preprocessing EEG data

In [ ]:
# function expects a raw object with .fif file ending
pp.prepro(
    study=2,
    trigger="stimulus",
    l_freq=1,
    h_freq=40,
    tmin=-1,
    tmax=1,
    resample_rate=250,
    sf=2500,
    detrend=1,
    ransac=1,
    autoreject=0,
)

# how many channels were interpolated?
pp.n_channels_interpolated(study=2, trigger="stimulus", l_freq=0.1)

# run ica on clean, epoched data
ica.run_ica(study=2, infomax=1, save_psd=1)
# correlate with EOG and ECG and mark bad componets for rejection

ica.label_ica_correlation(study=2)

# usa icalabel to mark components for rejection
# ica.label_iclabel(study=1)

#### ICA stats

In [ ]:
# which study to run the analysis on
study = 2

In [ ]:
# load the csv file that contains the number of components rejected
df_comp = pd.read_csv(
    Path("E:/expecon_ms/data/eeg/prepro_ica/clean_epochs_corr{study!s}/ica_components_stats_icacorr.csv")
)

# mean components rejected
print(f' on average {df_comp["0"].mean()} components were rejected')
print(f' the sdt of components rejected is {df_comp["0"].std()}')
print(f' the maximum of components rejected is {df_comp["0"].max()}')
print(f' the minimum of components rejected is {df_comp["0"].min()}')

### 3. Evoked potentials

In [ ]:
# compare evokeds and plot contrasts
evokeds = evo.create_contrast(
    study=2, drop_bads=True, laplace=False, subtract_evoked=False, save_data_to_disk=False, save_drop_log=False
)

In [ ]:
# plot evoked contrast and topography for the contrast
evo.plot_roi(study=2, data=evokeds, tmin=-0.9, tmax=0, tmin_base=-0.1, tmax_base=0)

### 4. Time-frequency analysis

In [ ]:
# compute tfr representations for each condition
tfr.compute_tfr(
    study=2,
    cond="probability",
    tmin=-0.7,
    tmax=0,
    fmax=35,
    fmin=3,
    laplace=False,
    induced=True,
    mirror=False,
    drop_bads=True,
)

#### stimulus probability contrast

In [ ]:
# load the tfr data for each condition for probability conds.
tfr_a_cond, tfr_b_cond = tfr.load_tfr_conds(
    studies=[1, 2],
    cond="probability",
    cond_a_name="high_-0.7_0_induced",
    cond_b_name="low_-0.7_0_induced",
    cond_a_names=["high_prevhit_-0.7_0_induced", "high_prevmiss_-0.7_0_induced", "high_prevcr_-0.7_0_induced"],
    cond_b_names=["low_prevhit_-0.7_0_induced", "low_prevmiss_-0.7_0_induced", "low_prevcr_-0.7_0_induced"],
)

In [ ]:
# run-cluster-based permutation tests for the conditions contrast
# and plot sign. cluster
tfr.plot_tfr_cluster_test_output(
    cond="probability",
    tfr_a_cond=tfr_a_cond,
    tfr_b_cond=tfr_b_cond,
    threed_test=False,
    cond_a_name="high_induced",
    cond_b_name="low_induced",
    channel_names=["CP4"],
)

previous response contrast

In [ ]:
# load the tfr data for each condition for prev_resp conds.
tfr_a_cond, tfr_b_cond = tfr.load_tfr_conds(
    studies=[1, 2],
    cond="prev_resp",
    cond_a_name="prevyesresp_highprob_prevstim_-0.7_0_induced",
    cond_b_name="prevnoresp_highprob_prevstim_-0.7_0_induced",
    cond_a_names=["prevyesresp_samecue_lowprob_-0.7_0_induced", "prevyesresp_samecue_highprob_-0.7_0_induced"],
    cond_b_names=["prevnoresp_samecue_lowprob_-0.7_0_induced", "prevnoresp_samecue_highprob_-0.7_0_induced"],
)

In [ ]:
# run cluster based permutation tests for the conditions contrasts
# and plot sign. cluster
tfr.plot_tfr_cluster_test_output(
    cond="prev_resp",
    tfr_a_cond=tfr_a_cond,
    tfr_b_cond=tfr_b_cond,
    threed_test=False,
    cond_a_name="prevyesresp",
    cond_b_name="prevnoresp",
    channel_names=["CP4"],
)

### 5. Source reconstruction

In [ ]:
# run source reconstruction for each condition
source_reco.run_source_reco(
    study=1,
    cond="prev_resp",
    mirror=False,
    dics=True,
    fmin=15,
    fmax=25,
    tmin=-0.7,
    tmax=-0.1,
    drop_bads=True,
    subtract_evokeds=True,
    plot_alignment=False,
)

In [ ]:
# plot source contrast (grand average over all participants)
# opens plots in separate windows
source_reco.plot_grand_average_source_contrast(study=1, cond="probability", method="beamformer", save_plots=False)

In [ ]:
# run source localization for each epoch based on filter from contrast

In [ ]:
source_reco.run_source_reco_per_trial(study=2, fmin=15, fmax=25, tmin=-0.7, tmax=-0.1, drop_bads=True)